# Experiment 5: Seasonality Features

Nachat Jatusripitak

In this notebook, we experiment with adding features to indicate seasonality, including monthly and daily features

In [ ]:
# Import required packages
import xgboost as xgb
import pandas as pd
import src.train_utils as T
import xarray as xr
import matplotlib.pyplot as plt

plt.style.use('ggplot')  

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', None)

In [ ]:
# Load dataset
ds = xr.open_dataset('../datasets/exp_4_ds.nc')

# Hold out 2022
mask = ds.time.dt.year < 2022
ds_excl_2022 = ds.sel(time=mask)

train_df = ds_excl_2022.to_dataframe().reset_index()

# Add day of week indicator (one-hot and integer)
train_df['day_of_week'] = train_df['time'].dt.dayofweek
dow_dummies = pd.get_dummies(train_df['day_of_week'], prefix='dow')
train_df = pd.concat([train_df, dow_dummies], axis=1)

# Add month indicator (one-hot and integer)
train_df['month'] = train_df['time'].dt.month
month_dummies = pd.get_dummies(train_df['month'], prefix='month')
train_df = pd.concat([train_df, month_dummies], axis=1)

# Add binary is_weekend flag
train_df['is_weekend'] = train_df['time'].dt.dayofweek >= 5
train_df['is_weekend'] = train_df['is_weekend'].astype(int)


## Run 1: Try all features

In [4]:
base = ['delta_pm25_t', 'delta_pm25_t-1', 'delta_pm25_t-2', 'delta_pm25_t-3', 'pm25_t', 'u_wind_t', 'v_wind_t',
       'dew_temp_t', 'temp_t', 'surf_pressure_t', 'precip_t',
       'frp_t', 'elevation_t', 'r_humidity_t']

base += ['delta_pm25_avg_3x3', 'delta_pm25_avg_5x5', 'delta_pm25_avg_7x7', 'delta_pm25_avg_9x9']

base += ['pm25_avg_3x3', 'pm25_avg_5x5', 'pm25_avg_7x7', 'pm25_avg_9x9']

base += ['frp_max_3x3', 'frp_max_5x5', 'frp_max_7x7', 'frp_max_9x9']

base += ['frp_buffer_sum_3x3', 'frp_buffer_sum_5x5', 'frp_buffer_sum_7x7', 'frp_buffer_sum_9x9']


params = {
    'max_depth': 4,            
    'learning_rate': 0.1,     
    'n_estimators': 150,     
    'subsample': 0.8,          
    'colsample_bytree': 0.8,   
    'objective': 'reg:pseudohubererror'   
}

feature_experiments = [
    ('base', base),
    ('base + is_weekend', base + ['is_weekend']),
    ('base + day_of_week', base + ['day_of_week']),
    ('base + dow_one_hot', base + ['dow_0', 'dow_1', 'dow_2', 'dow_3', 'dow_4', 'dow_5', 'dow_6']),
    ('base + month', base + ['month']),
    ('base + month_one_hot', base + ['month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'])
]

model=xgb.XGBRegressor(**params, random_state=191)

results = T.run_experiments(
    df=train_df, 
    model=model, 
    feature_experiments=feature_experiments, 
    train_days=365*2,
    gap_days=21,
    val_days=49
)

print(results.sort_values('mean_rmse'))

  0%|          | 0/6 [00:00<?, ?it/s]

Running experiment: base


  0%|          | 0/10 [00:00<?, ?it/s]

Running experiment: base + is_weekend


  0%|          | 0/10 [00:00<?, ?it/s]

...

Running experiment: base + month


  0%|          | 0/10 [00:00<?, ?it/s]

Running experiment: base + month_one_hot


  0%|          | 0/10 [00:00<?, ?it/s]

             experiment  n_features  mean_rmse  mean_mae  rmse_fold_1  rmse_fold_2  rmse_fold_3  rmse_fold_4  rmse_fold_5  rmse_fold_6  rmse_fold_7  rmse_fold_8  rmse_fold_9  rmse_fold_10
5  base + month_one_hot          42   4.221747  2.861813     1.637629     4.846499     4.975658     8.401618    12.723725     1.987511     0.907936     0.986088     1.807607      3.943198
4          base + month          31   4.230838  2.867593     1.625597     4.883335     4.965840     8.372669    12.844357     1.980262     0.907508     0.979888     1.813635      3.935289
1     base + is_weekend          31   4.248124  2.874278     1.624974     4.815865     5.023804     8.387665    12.950578     2.016352     0.905613     0.990801     1.817694      3.947898
0                  base          30   4.248382  2.882070     1.635246     4.808600     5.038874     8.413009    12.945456     1.995112     0.908033     0.986864     1.819594      3.933036
2    base + day_of_week          31   4.259099  2.882101    

In [1]:
df = results
# Sort by mean_rmse ascending
df_sorted = df.sort_values("mean_rmse", ascending=False)

# Display the sorted DataFrame (experiment and mean_rmse)
display_df = df_sorted[["experiment", "mean_rmse"]].copy()
display_df["mean_rmse"] = display_df["mean_rmse"].round(3)

print(display_df.to_string(index=False))

plt.figure(figsize=(10, 6))
bars = plt.bar(display_df["experiment"], display_df["mean_rmse"], color='#27aeef')
plt.ylim(display_df["mean_rmse"].min() - 0.01, display_df["mean_rmse"].max() + 0.01)
plt.xticks(rotation=15, ha="right")
plt.xlabel("Experiment", fontsize=12)
plt.ylabel("Mean RMSE", fontsize=12)

for bar, mean in zip(bars, display_df["mean_rmse"]):
    y = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2, y + 0.002, f"{mean:.3f}", 
             ha='center', va='bottom', fontsize=10)

plt.title('Mean RMSE vs. Seasonality Indicator')
plt.tight_layout()
plt.show()

NameError: name 'results' is not defined

## Run 2: Is it better to have month-specific indicators or just burn season indicators?

In [14]:
base = ['delta_pm25_t', 'delta_pm25_t-1', 'delta_pm25_t-2', 'delta_pm25_t-3', 'pm25_t', 'u_wind_t', 'v_wind_t',
       'dew_temp_t', 'temp_t', 'surf_pressure_t', 'precip_t',
       'frp_t', 'elevation_t', 'r_humidity_t']

base += ['delta_pm25_avg_3x3', 'delta_pm25_avg_5x5', 'delta_pm25_avg_7x7', 'delta_pm25_avg_9x9']

base += ['pm25_avg_3x3', 'pm25_avg_5x5', 'pm25_avg_7x7', 'pm25_avg_9x9']

base += ['frp_max_3x3', 'frp_max_5x5', 'frp_max_7x7', 'frp_max_9x9']

base += ['frp_buffer_sum_3x3', 'frp_buffer_sum_5x5', 'frp_buffer_sum_7x7', 'frp_buffer_sum_9x9']


params = {
    'max_depth': 4,            
    'learning_rate': 0.1,     
    'n_estimators': 150,     
    'subsample': 0.8,          
    'colsample_bytree': 0.8,   
    'objective': 'reg:pseudohubererror'   
}

feature_experiments = [
    ('base + burn season', base + ['month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_11', 'month_12']),
    ('base + month_one_hot', base + ['month_1', 'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10', 'month_11', 'month_12'])
]

model=xgb.XGBRegressor(**params, random_state=191)

results = T.run_experiments(
    df=train_df, 
    model=model, 
    feature_experiments=feature_experiments, 
    train_days=365*2,
    gap_days=21,
    val_days=49
)

print(results.sort_values('mean_rmse'))

  0%|          | 0/2 [00:00<?, ?it/s]

Running experiment: base + burn season


  0%|          | 0/10 [00:00<?, ?it/s]

Running experiment: base + month_one_hot


  0%|          | 0/10 [00:00<?, ?it/s]

             experiment  n_features  mean_rmse  mean_mae  rmse_fold_1  rmse_fold_2  rmse_fold_3  rmse_fold_4  rmse_fold_5  rmse_fold_6  rmse_fold_7  rmse_fold_8  rmse_fold_9  rmse_fold_10
1  base + month_one_hot          42   4.221747  2.861813     1.637629     4.846499     4.975658     8.401618    12.723725     1.987511     0.907936     0.986088     1.807607      3.943198
0    base + burn season          37   4.236164  2.872558     1.632492     4.822143     5.019202     8.404433    12.848427     1.976584     0.906787     0.981022     1.813469      3.957085


In [ ]:
# Export dataset with best features
months = ds['time'].dt.month

for m in range(1, 13):
    ds[f"month_{m:02d}"] = (months == m).astype(int)

ds.to_netcdf(
    "exp_5_ds.nc",
    format="NETCDF4",       
    engine="netcdf4",      
    encoding={
        var: {
            "zlib": True,
            "complevel": 4,
        }
        for var in ds.data_vars
    }
)